# Main


In [8]:
from optibook.synchronous_client import Exchange
%load_ext autoreload
%autoreload 2

from utils_3 import *

#import time
import logging
#import sys

logger = logging.getLogger('client')
logger.setLevel('ERROR')

print("Setup was successful.")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Setup was successful.


In [9]:
instrument_id = 'TEST'
Airbus = 'AIRBUS'
Allianz = 'ALLIANZ'
ASML = 'ASML'
LVMH = 'LVMH'
SAP = 'SAP'
Siemens = 'SIEMENS'
Total = 'TOTAL'
Unilever = 'UNILEVER'

#Airbus + Allianz = Pair_1

volume_storage = []
current_positions = []
new_positions = []

pair_list = []
levels_deep_list = []


### Connect to Exchange

In [10]:
e = Exchange()
a = e.connect()

stock_prices_max = {'AIRBUS': 22,
 'SIEMENS': 31,
 'TOTAL': 218,
 'UNILEVER': 358,
 'SAP': 100,
 'ASML': 30,
 'LVMH': 218,
 'ALLIANZ': 400}
stock_prices_min = {'AIRBUS': 21,
 'SIEMENS': 30,
 'TOTAL': 99,
 'UNILEVER': 163,
 'SAP': 40,
 'ASML': 15,
 'LVMH': 81,
 'ALLIANZ': 161}

# you can also define host/user/pass yourself
# when not defined, it is taken from ~/.optibook file if it exists
# if that file does not exists, an error is thrown

#e = Exchange(host='host-to-connect-to')
#a = e.connect(username='your-username', password='your-password')


2021-03-18 23:39:07,106 [asyncio   ] [MainThread  ] Using selector: EpollSelector


# Creates tables of p-values and hedge ratios for all stock pairs

In [4]:
#get_last_trade_value('TOTAL', 'bid', e)

data=data_loader()
companies = list(data.columns)[1:]

df_pvals, df_ratios = tabulate_cointeg_pvals_ratios(data, only_significant=0)
print(df_pvals)
print(df_ratios)





#exchange_test(e)
#trading_test('ALLIANZ','SAP', data)

             AIRBUS    SIEMENS      TOTAL   UNILEVER        SAP      ASML  \
AIRBUS     0.943388  0.0906278   0.132822    0.17386  0.0795951  0.139368   
SIEMENS    0.165182   0.962164  0.0196505  0.0737529   0.487771  0.506406   
TOTAL      0.438638  0.0449577   0.950746          0   0.723523  0.724601   
UNILEVER   0.953021   0.464585          0   0.942114   0.992589  0.989137   
SAP       0.0547673   0.234495   0.217012     0.2175   0.948142         0   
ASML       0.283943   0.614337   0.560743   0.561383          0  0.939931   
LVMH       0.664654   0.580269    0.60094   0.603265   0.610633  0.682753   
ALLIANZ    0.837602   0.658034   0.678152   0.646619    0.77882  0.840681   

              LVMH      ALLIANZ  
AIRBUS    0.365664     0.386927  
SIEMENS   0.486326     0.488133  
TOTAL      0.74926     0.750051  
UNILEVER   0.99281     0.990119  
SAP       0.231159     0.242959  
ASML       0.65218     0.672598  
LVMH      0.956065            0  
ALLIANZ          0  5.04689e-11  


# Batota scripts

In [4]:
comps = ["TOTAL", "UNILEVER", "SAP", "ASML", "LVMH", "ALLIANZ"]

2021-03-18 11:45:31,554 [client    ] [Thread-4    ] Forcing a disconnect due to an error: Closing connection due to blacklisting by a management client: not running.


In [5]:
for comp in comps:
    outstanding = e.get_outstanding_orders(comp)
    for o in outstanding.values():
        result = e.delete_order(comp, order_id=o.order_id)


NameError: name 'comps' is not defined

In [ ]:
for comp in comps:
    e.insert_order(comp, price=10.0, volume=80, side='bid', order_type='limit')
    time.sleep(1)
    e.insert_order(comp, price=490.0, volume=80, side='ask', order_type='limit')
    time.sleep(1)

In [ ]:
new_comp="UNILEVER"

e.insert_order(new_comp, price=10.0, volume=80, side='bid', order_type='limit')
time.sleep(1)
e.insert_order(new_comp, price=490.0, volume=80, side='ask', order_type='limit')
time.sleep(1)

# Extracts unique cointegrated stock pairs with associated c and gamma values

In [11]:
data=data_loader()
cointeg_comp_pairs = cointeg_significant_pairs(data, alpha=0.01)
print(cointeg_comp_pairs)

[('TOTAL', 'UNILEVER', -0.5008537179289747, 1.0001657849028531), ('SAP', 'ASML', -1.999879825520801, 1.9999539206651156), ('LVMH', 'ALLIANZ', 0.16680854548929716, 0.8333101843419742)]


# Test for logging trade ticks in a csv file

In [12]:
tick_logger_csv(data, e, overwrite=0)

1

# Definition of zt_trader

In [5]:
# Checks zt for a given stock pair and compares it to levels_deep for that pair, to determine whether we should execute a trade
# The function then executes that trade.
def zt_trader(stock_y, stock_x, c, gamma, levels_deep, current_positions, volume_storage, e, limits, base_volume):
    """
    limit_1 = limits[0]
    limit_2 = limits[1]
    limit_3 = limits[2]
    """
    base_volume = round(base_volume)
    
    res = 0
    
    #used_limit = abs(levels_deep)
    
    zt_up, zt_down, yt, xt = check_zt(stock_y, stock_x, c, gamma, e) # pulls the value of zt, as well as the stock prices used to calculate it

    shallower_critical_limit = limits[abs(levels_deep)-1]
    
    # Checks whether zt is below (current limit - 1), using different values for zt depending on whether an evaluation of TRUE would imply trading up (levels_deep increases) or trading down (levels_deep decreases).
    if (zt_down < shallower_critical_limit and levels_deep > 0) or (abs(zt_up) < shallower_critical_limit and levels_deep < 0) or (zt_down < 0 and levels_deep == 1) or (zt_up > 0 and levels_deep == -1):
        
        # trade shallower
        res = trade_shallower(stock_y, stock_x, levels_deep, volume_storage, e)
        print("stored volume using levels_deep =", levels_deep) # prints for debugging purposes
        
        volume_storage = store_volumes(stock_y, stock_x, abs(levels_deep), 0,0, volume_storage) # Resets the stored volume for this level
        current_positions = get_nr_positions_custom(e) # Stores our new actual positions
        # Needs to check whether we're actually at zero. Otherwise we need to do another trade.
        
        
        # Decrease abs(levels_deep) by 1
        if levels_deep > 0:
            levels_deep -= 1
        elif levels_deep < 0:
            levels_deep += 1
    
    
    elif abs(levels_deep) != len(limits)-1: # Checks that we are not at the deepest level
        if zt_up > 0 and abs(zt_up) > limits[abs(levels_deep)+1]: # Checks whether zt has exceeded a limit (except for the last one)
            
            # Checks whether we've already bought in at this limit
            for pair_table in volume_storage:
                if (pair_table[0] == [stock_y, stock_x]):
                    print(pair_table)
                    if pair_table[abs(levels_deep)+1] == [0, 0]:
                        
                        # Trade deeper up
                        res = trade_deeper(1, stock_y, stock_x, levels_deep, yt, xt, gamma, base_volume, e)
                        print(res)
                        levels_deep += 1
        
                        hedge_volume_to_store=res[1]
                        base_volume_to_store=res[2]
                        volume_storage = store_volumes(stock_y, stock_x, levels_deep, base_volume_to_store, hedge_volume_to_store, volume_storage) # Stores the traded volume
                        current_positions = get_nr_positions_custom(e) # Stores our new actual positions
        
        
        if zt_down < 0 and abs(zt_down) > limits[abs(levels_deep)+1]: # Checks whether zt has exceeded a limit (except for the last one)

            # Checks whether we've already bought in at this limit
            for pair_table in volume_storage:
                if (pair_table[0] == [stock_y, stock_x]):
                    print(pair_table)
                    if pair_table[abs(levels_deep)+1] == [0, 0]:

                        # Trade deeper down
                        res = trade_deeper(0, stock_y, stock_x, levels_deep, yt, xt, gamma, base_volume, e)
                        print(res)
                        levels_deep -= 1


                        hedge_volume_to_store=res[1]
                        base_volume_to_store=res[2]
                        volume_storage = store_volumes(stock_y, stock_x, levels_deep, base_volume_to_store, hedge_volume_to_store, volume_storage) # Stores the traded volume
                        current_positions = get_nr_positions_custom(e) # Stores our new actual positions
                        

    # prints for debugging purposes
    print(f"Now {levels_deep} levels deep on ", stock_y, stock_x)
    print(f"zt_up: {zt_up}, zt_down: {zt_down}")

    return(res, levels_deep, current_positions, volume_storage)

# Resets positions and volume storage

In [13]:
# Total reset of positions and storage-lists
# Run this before testing zt_trader
#limits = [0, 0.001, 0.002, 0.003, 0.004]

volume_storage = []
limits = [0, 0.001, 0.003, 0.004]
max_levels=len(limits)-1
base_volume=500/max_levels -1

hack_out_of_positions(e) # Resets all our our actual holdings to 0
current_positions = get_nr_positions_custom(e) # Resets our tracker of current positions
volume_storage = creates_volume_storage(cointeg_comp_pairs, max_levels) # Resets the storage for traded volumes at various levels

#
pair_list=creates_pairs_storage(cointeg_comp_pairs)
levels_deep_list=creates_levels_deep_storage(cointeg_comp_pairs)
gamma_list=create_gamma_list(cointeg_comp_pairs)
c_list=create_c_list(cointeg_comp_pairs)

res = 0
levels_deep = 0

{'TOTAL': -248, 'SAP': 0, 'ASML': 0, 'LVMH': 124, 'AIRBUS': 0, 'SIEMENS': 0, 'ALLIANZ': -50, 'UNILEVER': 150}
{'TOTAL': 0, 'SAP': 0, 'ASML': 0, 'LVMH': 0, 'AIRBUS': 0, 'SIEMENS': 0, 'ALLIANZ': 0, 'UNILEVER': 0}


# Runs the trading function

In [ ]:
# Tests zt_trader once
res, levels_deep, current_positions, volume_storage = zt_trader("TOTAL", "UNILEVER", -0.5, 1, levels_deep, current_positions, volume_storage, e, limits, base_volume=20)

In [13]:
# Test loop z-trader
time_var=0
while time_var==0:
    
    #trade_pair (1)
    #trade_pair (b)
    
    aux=0
    stock_A=pair_list[aux][0]
    stock_B=pair_list[aux][1]    

    res = 0
    res, levels_deep_list[aux], current_positions, volume_storage = zt_trader(stock_A, stock_B, c_list[aux], gamma_list[aux], levels_deep_list[aux], current_positions, volume_storage, e, limits, base_volume)


    # Check goal volumes vs actual positions
    volume_difs_1= check_volumes(stock_A, stock_B, max_levels, volume_storage, current_positions)
    while volume_difs_1 !=0:
        print ("VOLUME DIFF HERE")
        print(volume_difs_1)
        fix_res=fix_positions(stock_A, stock_B, volume_difs_1, e)
        current_positions = get_nr_positions_custom(e) 
        print ("FIXED volume diff. Res: ", fix_res)
        volume_difs_1= check_volumes(stock_A, stock_B, max_levels, volume_storage, current_positions)
        time.sleep(0.5)

        
    print("--------------------------------")
    time.sleep(0.5)

TypeError: zt_trader() missing 2 required positional arguments: 'stock_prices_max' and 'stock_prices_min'

2021-03-18 10:27:56,522 [client    ] [Thread-4    ] Forcing a disconnect due to an error: Closing connection because someone else logged in with the same credentials. Only one session may be active at the same time.


In [11]:
# Test loop z-trader
#MARIA'S TESTS
def automated_trading(current_positions,levels_deep_list,  volume_storage, pair_list, max_levels, limits, base_volume):
    time_var=0
    while time_var==0:
        aux=0
        for pair in pair_list:
            stock_A=pair_list[aux][0]
            stock_B=pair_list[aux][1]    

            res = 0
            res, levels_deep_list[aux], current_positions, volume_storage = zt_trader(stock_A, stock_B, c_list[aux], gamma_list[aux], levels_deep_list[aux], current_positions, volume_storage, e, limits, base_volume)


            # Check goal volumes vs actual positions
            volume_difs_1= check_volumes(stock_A, stock_B, max_levels, volume_storage, current_positions)
            while volume_difs_1 !=0:
                print ("VOLUME DIFF HERE")
                print(volume_difs_1)
                fix_res=fix_positions(stock_A, stock_B, volume_difs_1, e)
                current_positions = get_nr_positions_custom(e) 
                print ("FIXED volume diff. Res: ", fix_res)
                volume_difs_1= check_volumes(stock_A, stock_B, max_levels, volume_storage, current_positions)
                time.sleep(0.1)

            aux +=1
            print("--------------------------------")
            time.sleep(0.1)
    

In [10]:
# Runs the automated trader function for all stock pairs
automated_trading (current_positions,levels_deep_list,  volume_storage, pair_list, max_levels, limits, base_volume)

TypeError: automated_trading() missing 6 required positional arguments: 'aux', 'e', 'c_list', 'gamma_list', 'stock_prices_max', and 'stock_prices_min'

In [12]:
list_aux=[0, 1, 2]

tempori=0
while tempori ==0:
    for aux in list_aux:
        automated_trading(current_positions,levels_deep_list, volume_storage, pair_list, max_levels, limits, base_volume, aux, e, c_list, gamma_list, stock_prices_max, stock_prices_min)

Order UNILEVER V: 101 Side: Bid
Order TOTAL V: 166 Side: Ask
Order UNILEVER V: 101 Side: Bid
Order TOTAL V: 166 Side: Ask
(-114, 0)
(-114, 0)
(-114, 0)
(-114, 0)
(-114, 0)
(-114, 0)
(-114, 0)
(-114, 0)
(-114, 0)
(-114, 0)
(-114, 0)
(-114, 0)
(-94, 0)
(-94, 0)
(-94, 0)
(-94, 0)
(-94, 0)
Order TOTAL V: 166 Side: Bid
Order UNILEVER V: 101 Side: Ask
(144, -14)
(144, -14)
(144, 0)
(144, 0)
(144, 0)


KeyboardInterrupt: 

In [2]:
print(pair_list)
print (levels_deep_list)

NameError: name 'pair_list' is not defined

In [ ]:
print (volume_storage)
print (current_positions)
res=check_volumes('TOTAL', 'UNILEVER', max_levels, volume_storage, current_positions)
print ('Result:', res)

# Test loop the arbitrage trader function:

In [ ]:
# Test loop the z-trader function
# Before you begin, make sure all positions are at 0 and that you've set levels_deep = 0
time_var=0
while time_var==0:
    res, levels_deep = zt_trader(stock_y, stock_x, c, gamma, levels_deep, e, base_volume=20)
    time.sleep(0.5)

# Test for finding the current value of zt for a given stock pair

In [ ]:
# Checks what zt is for a given stock pair
print(check_zt("TOTAL", "UNILEVER", -0.5, 1, e))